In [4]:
# Import packages
import sys
sys.path.append('/home/juanfelipe/Desktop/research/kinetick/source')
import jax.numpy as np
from jax import grad, jit
from jax.scipy.optimize import minimize

import numpy as np
import matplotlib.pyplot as plt 
from tqdm import tqdm
#from sklearn.preprocessing import MinMaxScaler
from sklearn.kernel_ridge import KernelRidge
from parameter_learning import *
from plotlib import *
from models import * 
from interpolation import *
from kernels import *
from integrate import trap, trap2d
from sklearn.metrics import mean_squared_error
from jax import clear_caches
import jax.numpy as jnp
from jax import grad, jit
from jax.config import config; 
from numpy.linalg import norm
config.update("jax_enable_x64", True)

from functools import partial # for jit to make codes faster
clear_caches()

In [5]:
# True solution
def fp_sol(x,t):
    K = 2*(1-jnp.exp(-2*t))
    return 1/(K*jnp.pi)*jnp.exp(-(jnp.sum(x**2))/K)

fp_sol = vmap(fp_sol,in_axes=(0,None)) # vmap'ed

In [6]:
k = get_gaussianRBF(0.25)

In [7]:
# a in [-a,a]^2
a = 1
# \Omega
# n = 51
n = int(jnp.sqrt(500))
full_grid = jnp.linspace(-a,a,n)
interior_grid = full_grid[1:-1]
X,Y = jnp.meshgrid(*([interior_grid]*2))
interior_points = jnp.vstack([X.flatten(),Y.flatten()]).T
n_interior = len(interior_points)

# \partial\Omega
bottom = jnp.vstack([full_grid,-a*jnp.ones(n)]).T
top = jnp.vstack([full_grid,a*jnp.ones(n)]).T

left = jnp.vstack([-a*jnp.ones(n-2),interior_grid]).T
right = jnp.vstack([a*jnp.ones(n-2),interior_grid]).T

hor_points = jnp.vstack([bottom,top])
vert_points = jnp.vstack([left,right])

n_boundary = len(hor_points) + len(vert_points)

In [ ]:
def M(x):
    return jnp.exp(-0.5*(jnp.sum(x**2)))

# Elliptic operator
def P(k,index):
    def k_over_M(*args):
            return k(*args)/M(args[index])

    def Mgrad(*args):
         return M(args[index]) * jax.grad(k_over_M,argnums = index)(*args)
    def Pk(*args):
        return jnp.trace(jax.jacfwd(Mgrad)(*args))
    return Pk
# I - h*P  operator
h = 1e-2
def IminP(k,index):
    def op(*args):
        return k(*args) - h * P(k,index)(*args)
    return op
# Boundary operator: top and bottom
def BC_hor(k,index):
    def kM(*args):
        return k(*args)/M(args[index])
    def neumann_op_k(*args):
        return jnp.dot(jax.grad(kM,argnums=index)(*args),jnp.array([0,1.]))
    return neumann_op_k
# Boundary operator: left and right
def BC_vert(k,index):
    def kM(*args):
        return k(*args)/M(args[index])
    def neumann_op_k(*args):
        return jnp.dot(jax.grad(kM,argnums=index)(*args),jnp.array([1.,0]))
    return neumann_op_k
      